# ResNet模型

> 今天有空能够将resNet论文读一遍，同时使用Pytorch将ResNet模型实现

In [1]:
import torch
import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
from torchvision.datasets import CIFAR10

In [2]:
# 3x3 Convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)


# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3
                (self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [3]:
import torchvision.transforms as transforms
# Image Preprocessing
transform = transforms.Compose([
    transforms.Scale(40),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 Dataset
train_dataset = CIFAR10(root='data/cifar10',
                              train=True,
                              transform=transform,
                              download=False)

test_dataset = CIFAR10(root='data/cifar10',
                             train=False,
                             transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False)


In [6]:
resnet = ResNet(ResidualBlock,[2, 2, 2, 2])

In [13]:
# "一定要记着这样"
resnet.load_state_dict(torch.load('resnetmodel.pkl',map_location=lambda storage, loc: storage))

In [14]:
resnet.state_dict()

OrderedDict([('conv.weight', 
              (0 ,0 ,.,.) = 
               -0.2543  0.0832  0.0837
               -0.2695  0.0705  0.0461
               -0.0275  0.1106 -0.0640
              
              (0 ,1 ,.,.) = 
               -0.1218  0.1525  0.1290
               -0.2201  0.1923  0.2005
               -0.0854  0.1496 -0.0266
              
              (0 ,2 ,.,.) = 
               -0.2696 -0.1946  0.0418
               -0.0480  0.1863  0.1464
               -0.1042 -0.1077  0.2291
              
              (1 ,0 ,.,.) = 
               -0.0363 -0.2062  0.0619
                0.0677 -0.2740 -0.2559
               -0.1014  0.0332  0.1051
              
              (1 ,1 ,.,.) = 
                0.0775  0.2515 -0.0663
               -0.1249  0.2760  0.0521
                0.1185  0.3210  0.2663
              
              (1 ,2 ,.,.) = 
                0.1164 -0.1892  0.0699
               -0.0336 -0.0644 -0.1017
               -0.1608 -0.0550 -0.1644
              
    

In [15]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = resnet(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

In [16]:
print(correct/100)

86.2
